In [8]:
import pandas as pd
import numpy as np
import random
from os import listdir
from os.path import isfile, join,getsize,isdir
import csv
import numpy
import cPickle as pickle
import json
from random import shuffle
import warnings
import time
#import msgpack
import re
import matplotlib.pyplot as plt
from datetime import date
import math
import pylab
import pandas_profiling
import unicodedata

#n'affiche pas les warnings
warnings.filterwarnings('ignore')
#affiche la totalité des colonnes du dataframe
pd.set_option('display.max_columns', None)
#path = "C:\Users\Data Science 5\Desktop\L_equipe"
path = "W:\L_equipe"


### chargement des dataframes

In [3]:
#df_client = pd.read_pickle(r"W:\L_equipe\s_dataframe_general_xiti_clients.pkl")
#df_prod42 = pd.read_pickle(path+"\s_dataframe_general_vel_PROD42.pkl")
#df_client_clean = pd.read_pickle(path+"\s_dataframe_client_clean_final.pkl")
df_client = pd.read_pickle(r"W:\L_equipe\df_client_true_site.pkl")
#dataframe = pd.read_pickle(r"C:\Users\Data Science 5\Desktop\L_equipe\df_client_true_site.pkl")

#df_compte = pd.read_pickle(r"W:\L_equipe\s_dataframe_general_vel_compte.pkl")
#df_commande = pd.read_pickle(r"W:\L_equipe\s_dataframe_general_vel_commande.pkl")
#df_prod42_clean = pd.read_pickle(path+"\s_dataframe_prod42_clean.pkl")
#df_compte_clean = pd.read_pickle(r"W:\L_equipe\s_dataframe_compte_clean.pkl")
#df_commande_clean = pd.read_pickle(r"W:\L_equipe\s_dataframe_commande_clean.pkl")
#df_pages_juin_p1 = pd.read_pickle(r"W:\L_equipe\s_dataframe_general_xiti_pages_juin2015_partie1.pkl")
#df_pages_juin_p1 = pd.read_pickle("s_dataframe_pages_juin_partie1_clean.pkl")
#df_session_juin = pd.read_pickle(path+"\s_dataframe_general_xiti_sessions_juin2015.pkl")
#df_session_juin_clean = pd.read_pickle(r"W:\L_equipe\s_dataframe_session_juin2015_clean.pkl")

### Clean sites

In [5]:
#on ne garde que les sites de l'équipe et les client qui sont dans la base Client
#----------------------------------Sites de l'equipe ----------------------------------
start_time = time.clock()
df_client = pd.read_pickle(r"W:\L_equipe\df_client_true_site.pkl")

def nettoyage_site(dataframe) :
    dataframe = dataframe.loc[dataframe['id_site'].isin([492987,496306,496307,496838,502193,509043,539121,548647])]
    liste_client = dataframe['id_client'].tolist()
    return dataframe, liste_client 


df_client, liste_client = nettoyage_site(df_client)
print time.clock() - start_time, 'seconds'
#save
#df_client.to_pickle(r"C:\Users\Data Science 5\Desktop\L_equipe\df_client_true_site.pkl")

205.848806024 seconds


### Clean subs

In [7]:
#--------------------nettoyage nouveaux abonnés--------------------------

df_prod42 = pd.read_pickle(path+"\dataframes\s_dataframe_general_vel_PROD42.pkl")

def nettoyage_sub(dataframe, liste_client) :
    #selection des abonnés de l'equipe
    dataframe = dataframe.loc[dataframe['ClientUserId'].isin(liste_client)]
    #formatage datetime 
    dataframe['SubscriptionCreated'] = pd.to_datetime(dataframe['SubscriptionCreated'], format='%d/%m/%Y %H:%M:%S')
    dataframe['SubscriptionLastUpdated'] = pd.to_datetime(dataframe['SubscriptionLastUpdated'], format='%d/%m/%Y %H:%M:%S')
    dataframe['ServiceExpiry'] = pd.to_datetime(dataframe['ServiceExpiry'], format='%d/%m/%Y %H:%M:%S')
    
    #on ne garde que les abonnés de l'année 2015
    dataframe['anciennete_souscripteur'] = dataframe.groupby('ClientUserId').SubscriptionCreated.transform('min')
    dataframe = dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,1,1)) & (dataframe.anciennete_souscripteur < date(2015,7,1)) ]
    #de plus on ne prends pas en compte les mises a jour de juillet 2015
    dataframe = dataframe.loc[(dataframe.SubscriptionLastUpdated > date(2015,1,1)) & (dataframe.SubscriptionLastUpdated < date(2015,7,1))]
    
    #correction fautes de frapes et autres modifs
    dataframe.loc[(dataframe.ServiceTitle == 'SFR_Full_Extra_Suplementaire'), 'ServiceTitle']= 'SFR_Full_Extra_Supplementaire'
    dataframe.loc[(dataframe.ServiceTitle == 'Fete_des_peres_2015_v2'), 'ServiceTitle']= 'Fete_des_peres_2015'
    dataframe.loc[(dataframe.ServiceTitle == 'Running_heroes_1M_gratuit'), 'ServiceTitle']= 'Running_Heroes_1M_gratuit'
    dataframe.loc[(dataframe.ServiceTitle == 'TEST_ROLAND_GARROS_'), 'ServiceTitle']= 'RG2015_1€_15jrs_puis_11.99€'
    dataframe.loc[(dataframe.ServiceTitle == 'TEMPLATE_ROLAND_GARROS'), 'ServiceTitle']= 'RG2015_1€_15jrs_puis_11.99€'
    dataframe.loc[(dataframe.ServiceTitle == 'CocaCola_Total_1mois_essai '), 'ServiceTitle']= 'CocaCola_Total_1mois_essai'
    
    #fusion du nettoyage des souscriptions avec les abonnés
    df_sous = pd.read_excel(path+'\souscription_detail.xlsx')
    dataframe = pd.merge(dataframe, df_sous, how='inner', on=['ServiceID', 'ServiceID'])
    
    #group by
    #dataframe['somme_prix'] = dataframe.groupby(['ClientUserId', 'SubscriptionId'])['ExplicitPrice'].transform('sum')
    dataframe['SubscriptionCreated'] = dataframe.groupby(['ClientUserId','SubscriptionId'])["SubscriptionCreated"].transform('min')
    dataframe['SubscriptionLastUpdated'] = dataframe.groupby(['ClientUserId', 'SubscriptionId'])["SubscriptionLastUpdated"].transform('max')
    dataframe['ServiceExpiry'] = dataframe.groupby(['ClientUserId', 'SubscriptionId'])['ServiceExpiry'].transform('min')
    
    #feature generation
    dataframe = dataframe.loc[((dataframe.ServiceGroupTitle == 'Abo EQP') | (dataframe.ServiceGroupTitle == "Abo EQP via Tiers"))]
    dataframe.loc[((dataframe['ServiceGroupTitle'] == 'Abo EQP') & (dataframe.prix_mensuel != 0)), 'service'] = 'Abo_Equipe_payant' 
    dataframe.loc[((dataframe['ServiceGroupTitle'] == 'Abo EQP') & (dataframe.prix_mensuel == 0)), 'service'] = 'Abo_Equipe_gratuit'
    dataframe.loc[((dataframe['ServiceGroupTitle'] == 'Abo EQP via Tiers') & (dataframe.prix_mensuel != 0)), 'service'] = 'Abo_via_tiers_payant' 
    dataframe.loc[((dataframe['ServiceGroupTitle'] == 'Abo EQP via Tiers') & (dataframe.prix_mensuel == 0)), 'service'] = 'Abo_via_tiers_gratuit' 
          
    dataframe['provider'] = dataframe['UKI'].str.extract(('.*@(.+)\..*'))
    
    
    dataframe["sub_month"] = numpy.nan
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,1,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,2,1)), 'sub_month'] = '1'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,2,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,3,1)), 'sub_month'] = '2'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,3,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,4,1)), 'sub_month'] = '3'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,4,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,5,1)), 'sub_month'] = '4'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,5,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,6,1)), 'sub_month'] = '5'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,6,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,7,1)), 'sub_month'] = '6'
        
    dataframe["anciennete_souscripteur"] = (date(2015, 7, 1) - dataframe["anciennete_souscripteur"])
    dataframe['duree_abonnement_reelle'] = dataframe['ServiceExpiry'] - dataframe['SubscriptionCreated']
    dataframe["derniere_maj"] = (date(2015, 7, 1) - dataframe["SubscriptionLastUpdated"])
    dataframe.anciennete_souscripteur= pd.to_numeric(dataframe.anciennete_souscripteur)/(1000000000 * 60 * 60 * 24)
    dataframe.derniere_maj = pd.to_numeric(dataframe.derniere_maj)/(1000000000 * 60 * 60 * 24)
    dataframe.duree_abonnement_reelle = pd.to_numeric(dataframe.duree_abonnement_reelle)/(1000000000 * 60 * 60 * 24)
    
    dataframe['somme_mensualites'] = ((np.maximum(0,dataframe['duree_abonnement_reelle']
                                                  -dataframe["duree_cadeau"])/30.416)*dataframe["prix_mensuel"]
                                     + (dataframe["duree_cadeau"]/30.416)*dataframe["prix_mensuel_cadeau"])
    
    #sauvegarde d'un temp pour la data vis
    dataframe.to_pickle(path+"\dataframes\s_dataframe_temp_sub_2015.pkl")
    
    #dummies 
    dataframe = pd.get_dummies(dataframe, columns = ['SubscriptionStatus'], prefix ='SubStatus' )
    dataframe = pd.get_dummies(dataframe, columns = ['AutoRenew'], prefix ='AutoRenew' )
    dataframe = pd.get_dummies(dataframe, columns = ['service'], prefix = 'service')
    dataframe = pd.get_dummies(dataframe , columns=['sub_month'], prefix = 'sub_month')
    
    #group by pour traiter certaines dummies
    dataframe.rename(columns=lambda x: x.replace(" ", "_"), inplace=True)
    dataframe['SubStatus_Active_Subscription'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["SubStatus_Active_Subscription"].transform('min')
    dataframe['SubStatus_Cancelled_By_AutoRenew_Process'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["SubStatus_Cancelled_By_AutoRenew_Process"].transform('max')
    dataframe['SubStatus_Cancelled_By_Customer_Support_Agent'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["SubStatus_Cancelled_By_Customer_Support_Agent"].transform('max')
    dataframe['SubStatus_Cancelled_By_User'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["SubStatus_Cancelled_By_User"].transform('max')
    dataframe['SubStatus_Expired_Subscription'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["SubStatus_Expired_Subscription"].transform('max')
    dataframe['SubStatus_Failure_Retry_Mode'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["SubStatus_Failure_Retry_Mode"].transform('max')
    dataframe['duree_abonnement_reelle'] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["duree_abonnement_reelle"].transform('max')
    dataframe["AutoRenew_0.0"] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["AutoRenew_0.0"].transform('max')
    dataframe["AutoRenew_1.0"] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["AutoRenew_1.0"].transform('max')
    dataframe["AutoRenew_2.0"] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["AutoRenew_2.0"].transform('max')
    dataframe["AutoRenew_4.0"] = dataframe.groupby(
        ['ClientUserId','SubscriptionId'])["AutoRenew_4.0"].transform('max')
    
    #drop
    dataframe.drop(['ExplicitCurrency', 'PriceBandPaymentType', 'PriceBandAmount', 'PriceBandCurrency', 'IsTrial'
                  ,'ExplicitPaymentType',"ServiceGroup", 'ServiceGroupDescription',"UKI"
                      ,"SubscriptionStatusID","ServiceDescription","ServiceExpiry",'ServiceGroupTitle',"SubscriptionCreated" ,"ExplicitPrice"
                   ,"SubscriptionLastUpdated", 'ServiceTitle', 'cadeau',], axis=1, inplace=True)
    
    dataframe = dataframe.drop_duplicates()
    return dataframe

df_prod42_test = nettoyage_sub(df_prod42, liste_client)
#print df_prod42_test.somme_mensualites.sum()
#df_prod42_test.sort_values(by=['ClientUserId'])
#sauvegarde 
df_prod42_test.to_pickle(path+"\dataframes\s_dataframe_final_sub_2015.pkl")

IOError: [Errno 2] No such file or directory: 'W:\\L_equipe\\dataframes\\\\dataframes\\s_dataframe_general_vel_PROD42.pkl'

### Selection de la population des subscribers (à compléter)

In [2]:
#----------------------nouveaux abonnés juin------------------------
df_prod42_test = pd.read_pickle(path+"\dataframes\s_dataframe_final_sub_2015.pkl")
#l_prod42 = df_prod42_test.ClientUserId.unique()

#sub juin 
nouveaux_sub_juin = df_prod42_test.loc[df_prod42_test.sub_month_6 == 1]
nouveaux_sub_juin = nouveaux_sub_juin.loc[nouveaux_sub_juin.somme_mensualites >0]
nouveaux_sub_juin.drop(['sub_month_1', 'sub_month_2', 'sub_month_3'
            , 'sub_month_4','sub_month_5',"sub_month_6"], axis=1, inplace=True)

nouveaux_sub_juin.to_pickle(path+"\dataframes\s_dataframe_final_sub_payant_juin_2015.pkl")
#
#print nouveaux_sub_juin.shape #-> (2021, 28)
#print len (nouveaux_sub_juin.ClientUserId.unique()) #-> 2011
#
#test1 = nouveaux_sub_juin.loc[((nouveaux_sub_juin.service_Abo_Equipe_payant ==1 ) | (nouveaux_sub_juin.service_Abo_via_tiers_payant == 1))]
#print test1.shape #-> (946, 28)
#print len(test1.ClientUserId.unique()) #-> 945
#
#test2 = nouveaux_sub_juin.loc[nouveaux_sub_juin.somme_mensualites >0]
#print test2.shape #-> (439, 28)
#print len(test2.ClientUserId.unique()) #-> 439
## ***.to_pickle(path+"\s_final_groupby_dataframe_sub_juin_2015.pkl")

### Clean Clients 

In [9]:
#--------------------nettoyage clients--------------------------

dataframe = pd.read_pickle(r"W:\L_equipe\df_client_true_site.pkl")

dataframe = dataframe.loc[dataframe['id_client'].isin(liste_client)]
#formatage datetime 
dataframe['premiere_visite'] = pd.to_datetime(dataframe['premiere_visite'], format='%Y-%m-%d %H:%M:%S')
dataframe['derniere_visite'] = pd.to_datetime(dataframe['derniere_visite'], format='%Y-%m-%d %H:%M:%S')
#group by
dataframe['somme_visites_totale'] = dataframe.groupby(['id_client']).visites.transform('sum')
dataframe['somme_pages_vues_totale'] = dataframe.groupby(["id_client"])["pages_vues"].transform('sum')
dataframe['anciennete'] = dataframe.groupby(['id_client'])["premiere_visite"].transform('min')
dataframe['recence_de_visite'] = dataframe.groupby(['id_client'])["derniere_visite"].transform('max')
    
#on choisit les clients qui on eu une acitivité entre  mars et mai 2015
dataframe = dataframe.loc[(dataframe.anciennete < date(2015,6,1)) & (dataframe.recence_de_visite > date(2015,2,28))]

dataframe['recence_de_visite'] =(date(2015, 7, 1) - dataframe.recence_de_visite)
dataframe.recence_de_visite = pd.to_numeric(dataframe.recence_de_visite)/(1000000000 * 60 * 60 * 24)
dataframe['anciennete'] =(date(2015, 7, 1) - dataframe.anciennete)
dataframe.anciennete = pd.to_numeric(dataframe.anciennete)/(1000000000 * 60 * 60 * 24)

dataframe['somme_visites_par_site'] = dataframe.groupby(["id_client","id_site"])["visites"].transform('sum')
dataframe['pourcentage_visites_par_site'] = dataframe["somme_visites_par_site"]/dataframe["somme_visites_totale"]*100

dataframe['somme_pages_vues_par_site'] = dataframe.groupby(["id_client","id_site"])["pages_vues"].transform('sum')
dataframe['pourcentage_pages_vues_par_site'] = dataframe["somme_pages_vues_par_site"]/dataframe["somme_pages_vues_totale"]*100

#dummies
dataframe["id_site2"] = dataframe.id_site
dataframe = pd.get_dummies(dataframe, columns =['id_site'], prefix ='%visite_site')
dataframe = pd.get_dummies(dataframe, columns =['id_site2'], prefix ='%pages_vues_site')

#mutliplication des dummies pour optenir le pourcentage de visites par site
dataframe[['%visite_site_492987','%visite_site_496306','%visite_site_496307','%visite_site_496838','%visite_site_502193'
,'%visite_site_509043','%visite_site_539121','%visite_site_548647']] = dataframe[['%visite_site_492987','%visite_site_496306'
,'%visite_site_496307','%visite_site_496838','%visite_site_502193','%visite_site_509043','%visite_site_539121'
,'%visite_site_548647']].multiply(dataframe["pourcentage_visites_par_site"], axis="index")

#mutliplication des dummies pour optenir le pourcentage de pages vues par site
dataframe[['%pages_vues_site_492987','%pages_vues_site_496306','%pages_vues_site_496307','%pages_vues_site_496838'
,'%pages_vues_site_502193','%pages_vues_site_509043','%pages_vues_site_539121'
,'%pages_vues_site_548647']] = dataframe[['%pages_vues_site_492987','%pages_vues_site_496306','%pages_vues_site_496307'
,'%pages_vues_site_496838','%pages_vues_site_502193','%pages_vues_site_509043','%pages_vues_site_539121'
,'%pages_vues_site_548647']].multiply(dataframe["pourcentage_pages_vues_par_site"], axis="index")

dataframe[['%visite_site_492987','%visite_site_496306','%visite_site_496307','%visite_site_496838','%visite_site_502193'
,'%visite_site_509043','%visite_site_539121','%visite_site_548647']] = dataframe.groupby(["id_client"])['%visite_site_492987'
,'%visite_site_496306','%visite_site_496307','%visite_site_496838','%visite_site_502193','%visite_site_509043'
,'%visite_site_539121','%visite_site_548647'].transform('max')

dataframe[['%pages_vues_site_492987','%pages_vues_site_496306','%pages_vues_site_496307','%pages_vues_site_496838'
,'%pages_vues_site_502193','%pages_vues_site_509043','%pages_vues_site_539121'
,'%pages_vues_site_548647']] = dataframe.groupby(["id_client"])['%pages_vues_site_492987','%pages_vues_site_496306'
,'%pages_vues_site_496307','%pages_vues_site_496838','%pages_vues_site_502193','%pages_vues_site_509043'
,'%pages_vues_site_539121','%pages_vues_site_548647'].transform("max")


dataframe.drop(['somme_visites_par_site', 'somme_pages_vues_par_site','premiere_visite', 'derniere_visite'
                , 'fileid', 'visites', 'pages_vues', 'pourcentage_pages_vues_par_site','pourcentage_visites_par_site']
               , axis=1, inplace=True)
dataframe = dataframe.drop_duplicates()

dataframe.to_pickle(path+"\dataframes\s_dataframe_final_clients_mam_pas_forcmnt_juin.pkl")

#client ayant fait au moins une visite entre mars et mai 2015 et au moins une en juin 2015
dataframe = dataframe.loc[dataframe.recence_de_visite < 31]
dataframe.to_pickle(path+"\dataframes\s_dataframe_final_clients.pkl")

#df_client_clean = pd.read_pickle(path+"\dataframes\s_dataframes_final_clients.pkl")

### Clean commmandes 

In [ ]:
dataframe= pd.read_pickle(path+"\dataframes\s_dataframe_general_vel_commande.pkl")

def nettoyage_commande(dataframe, liste_client) :
    #selection des abonnés de l'equipe
    dataframe = dataframe.loc[dataframe['ClientUserId'].isin(liste_client)]
    #formatage datetime 
    dataframe['OrderDate'] = pd.to_datetime(dataframe['OrderDate'], format='%d/%m/%Y %H:%M:%S')

    #On ne considère pas les achats et les souscriptions post juin 2015
    dataframe = dataframe.loc[dataframe.OrderDate < date(2015,7,1)]
    
    dataframe = dataframe.loc[dataframe.OrderStatus == 'Completed']
    #Pour bien faire le lien avec la table souscription on réccupère les nouveaux abonnés 2015
    datasub = dataframe.loc[dataframe.ProductType == 'Service']
    datasub['anciennete_souscripteur'] = datasub.groupby(['ClientUserId']).OrderDate.transform('min')
    datasub = datasub.loc[((datasub.anciennete_souscripteur > date(2015,1,1)) & (datasub.OrderDate < date(2015,7,1))) ]
    l_sub = datasub.ClientUserId.unique()
    datasub = datasub[["ClientUserId", "anciennete_souscripteur"]]
    datasub = datasub.drop_duplicates()

    dataframe = dataframe.merge(datasub,how='left', left_on='ClientUserId', right_on='ClientUserId')
    
    dataframe["sub_month"] = numpy.nan
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,1,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,2,1)), 'sub_month'] = '1'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,2,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,3,1)), 'sub_month'] = '2'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,3,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,4,1)), 'sub_month'] = '3'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,4,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,5,1)), 'sub_month'] = '4'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,5,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,6,1)), 'sub_month'] = '5'
    dataframe.loc[(dataframe.anciennete_souscripteur > date(2015,6,1)) 
                  & (dataframe.anciennete_souscripteur < date(2015,7,1)), 'sub_month'] = '6' 
    
    #group by
    
    dataframe['recence_souscripteur'] = 0
    dataframe['recence_souscripteur'].loc[dataframe.ProductType == 'Service'] =\
    dataframe.loc[dataframe.ProductType == 'Service'].groupby(['ClientUserId'])["OrderDate"].transform('max')
    dataframe["recence_souscripteur"].loc[dataframe.ProductType == 'Service'] = (date(2015, 7, 1) - dataframe.recence_souscripteur)
    dataframe.recence_souscripteur = pd.to_numeric(dataframe.recence_souscripteur)/(1000000000 * 60 * 60 * 24)
    
    dataframe.anciennete_souscripteur = (date(2015, 7, 1) - dataframe.anciennete_souscripteur)
    dataframe.anciennete_souscripteur = pd.to_numeric(dataframe.anciennete_souscripteur)/(1000000000 * 60 * 60 * 24)
    
    dataframe["recence_article"] = 0
    dataframe['recence_article'].loc[dataframe.ProductType == 'Digital'] =\
    dataframe.loc[dataframe.ProductType == 'Digital'].groupby(['ClientUserId'])["OrderDate"].transform('max')
    dataframe["recence_article"].loc[dataframe.ProductType == 'Digital'] = (date(2015, 7, 1) - dataframe.recence_article)
    dataframe.recence_article = pd.to_numeric(dataframe.recence_article)/(1000000000 * 60 * 60 * 24)

    dataframe["anciennete_article"] = 0
    dataframe['anciennete_article'].loc[dataframe.ProductType == "Digital"] =\
    dataframe.loc[dataframe.ProductType == "Digital"].groupby(['ClientUserId'])["OrderDate"].transform('min')
    dataframe.anciennete_article.loc[dataframe.ProductType == "Digital"] = (date(2015, 7, 1) - dataframe.anciennete_article)
    dataframe.anciennete_article = pd.to_numeric(dataframe.anciennete_article)/(1000000000 * 60 * 60 * 24)
    
    dataframe["somme_paiement_par_abonnement"] =0
    dataframe["somme_paiement_par_abonnement"].loc[dataframe.ProductType == 'Service'] = \
    dataframe.loc[dataframe.ProductType == 'Service'].groupby(["ClientUserId","ServiceId"]).GrossAmount.transform('sum')
    
    dataframe['nb_mensualites'] = 0
    dataframe['nb_mensualites'].loc[dataframe.ProductType == 'Service'] =\
    dataframe.loc[dataframe.ProductType == 'Service'].groupby(["ClientUserId","ServiceId"]).BasketNumber.transform('count')
     

    dataframe["somme_mensualites"] = 0
    dataframe["somme_mensualites"].loc[dataframe.ProductType == 'Service'] = \
    dataframe.loc[dataframe.ProductType == 'Service'].groupby(["ClientUserId"]).BasketNumber.transform('count')
    
    
    dataframe["mensualite_moyenne"] = 0
    dataframe["mensualite_moyenne"].loc[dataframe.ProductType == 'Service'] = \
    dataframe.loc[dataframe.ProductType == 'Service'].groupby(['ClientUserId']).GrossAmount.transform("sum")
    dataframe["mensualite_moyenne"] = dataframe.mensualite_moyenne / dataframe.somme_mensualites
    
    dataframe['somme_paiement_totale'] = dataframe.groupby(["ClientUserId"]).GrossAmount.transform('sum')
    
    dataframe["somme_paiement_article"] =0
    dataframe["somme_paiement_article"].loc[dataframe.ProductType == 'Digital'] = \
    dataframe.loc[dataframe.ProductType == 'Digital'].groupby(["ClientUserId"]).GrossAmount.transform('sum')
        
    #features
    dataframe.ServiceId.fillna(0, inplace=True)
    #dataframe['theme_article'] = dataframe['ARTICLE_URL'].str.extract(('.*www.lequipe.fr/([a-zA-Z]+)/.*'))
    #dataframe.loc[dataframe.somme_paiement_par_abonnement == 0, 'commande_payante']=1
    #dataframe.loc[dataframe.somme_paiement_par_abonnement != 0, "commande_payante"]=0
    
    
    #dummies
    dataframe = pd.get_dummies(dataframe, columns = ['ProductType'], prefix ='prod_type')
    dataframe = pd.get_dummies(dataframe , columns=['sub_month'], prefix = 'sub_month')

    
    #dataframe['theme_article'] = dataframe['ARTICLE_URL'].str.extract(('.*www.lequipe.fr/([a-zA-Z]+)/.*'))
    #dataframe = pd.get_dummies(dataframe, columns=['theme_article'], prefix='theme_article')
    
    #duplication des infos utiles pour split abonnements et articles
    
    
    dataframe['nb_articles'] = dataframe.prod_type_Digital
    dataframe['nb_articles'] = dataframe.groupby(['ClientUserId']).nb_articles.transform("sum")
   
    #drop
    dataframe.drop(["AffiliateId","BasketNumber","Country","County","Currency","District","NetAmount","HouseFlatNumber",
        "HouseName","Message","Name","OfferId","OrderStatus","ParameterOrderId","PaymentMethod","PostCode","ProductDataSource",
        "Street","SupplierID","ThirdPartyRef","Titre","TotalTaxPercentage","TownCity","VATAmount","detail_mode_paiement","gift_id",
        "identifiant_commercial","identifiant_salon","lien","provenance","type",'ActivationCode',
        'DeliveryComments', 'subscriptionId','ContentItemId', "ARTICLE_TITLE", "ARTICLE_URL"
       ,'GrossAmount', 'OrderID', 'AccountID', "MPPGiftCode","Description", 'MPPGiftID', 'OrderDate', 'somme_paiement_par_abonnement',
        'nb_mensualites', 'ServiceId'], axis=1, inplace=True)
    
    dataframe = dataframe.drop_duplicates()
    dataframe["nb_abonnements"] = dataframe.groupby(["ClientUserId"]).prod_type_Service.transform("sum")
    
    #dataframe[['anciennete_article','anciennete_souscripteur', "somme_mensualites", 'mensualite_moyenne'
    #          , 'somme_paiement_totale', 'somme_paiement_article', 'prod_type_Digital', 'prod_type_Service'
    #          , 'sub_month_4','sub_month_5','sub_month_6', 'nb_abonnements'
    #          , 'nb_articles']] = dataframe.groupby(["ClientUserId"])['anciennete_article','anciennete_souscripteur'
    #          , "somme_mensualites", 'mensualite_moyenne', 'somme_paiement_totale', 'somme_paiement_article'
    #          , 'prod_type_Digital', 'prod_type_Service', 'sub_month_4','sub_month_5','sub_month_6'
    #          , 'nb_abonnements', 'nb_articles'].transform('max')
    #
    dataframe[['recence_article', 'recence_souscripteur']] = dataframe.groupby(['ClientUserId'])['recence_article'
                    , 'recence_souscripteur'].transform('min')
    dataframe = dataframe.drop_duplicates()
    dataframe.fillna(0, inplace=True)
    return dataframe

df_commande = nettoyage_commande(dataframe, liste_client)

#df_commande.sort_values(by='ClientUserId')
#df_commande.nb_abonnements.value_counts(dropna=False)
#pandas_profiling.ProfileReport(df_commande)

### Clean pages premiere partie (1iere étape terminée, deuxieme étape en cours)

In [3]:
#création de la liste de la popolation a sélectionner dans page et sessions
df = pd.read_pickle(path+"\dataframes\s_dataframe_final_clients.pkl")
df1 = pd.read_pickle(path+"\dataframes\s_dataframe_final_sub_payant_juin_2015.pkl")
liste_pop = df.id_client.unique().tolist()
liste_sub_juin = df1.ClientUserId.unique().tolist()
set_pop=set(liste_pop)
set_sub_juin = set(liste_sub_juin)
set_pop.update(set_sub_juin)  #liste d'id unique contenant notre population

In [6]:
#premiere étape clean chaque fichiers page et fichier session avec le cut sur notre population 
path = "W:\L_equipe\dataframes\\"
files  = []
files.extend([join(path,f) for f in listdir(path) if isfile(join(path, f)) 
                        if f.startswith('s_dataframe_general_xiti_pages') if getsize(join(path,f))>0])

for f in files :
    reg = re.match(('.*pages_(.*)'), f)
    dataframe = pd.read_pickle(f)
    dataframe = dataframe.loc[dataframe.id_site.isin([492987,496306,496307,496838,502193,509043,539121,548647])]
    dataframe = dataframe.loc[dataframe.id_client.isin(set_pop)]
    dataframe.to_pickle(path+"s_dataframe_temp_pages_payant_"+reg.group(1))

In [ ]:
#deuxieme étape (à faire car le cut n'est pas suffisant) on group by et on refera une étape de groupby par la suite
path = "W:\L_equipe\dataframes\\"
files  = []
files.extend([join(path,f) for f in listdir(path) if isfile(join(path, f)) 
                        if f.startswith('s_dataframe_temp_pages') if getsize(join(path,f))>0])
for f in files :
    dataframe.drop['index', axis=1, inplace=True]
    dataframe = pd.read_pickle(f)
    pd.get_dummies(dataframe, columns=["id_site"], prefix="id_site")
    #normalisation de niveau_2
    dataframe.niveau_2 = dataframe.niveau_2.str.lower()
    dataframe.niveau_2 = dataframe.niveau_2.str.normalize('NFKD')
    dataframe.niveau_2 = dataframe.niveau_2.str.encode('ASCII', 'ignore')


In [3]:
df = pd.read_pickle(path+"\dataframes\s_dataframe_temp_pagesjuin2015_partie1.pkl")

In [9]:
#df.drop(["index"],axis=1,inplace = True)
#df.sort_values(by=['id_session','id_client'])
df['theme_page1'] = df['url'].str.extract(("([a-zA-Z1-9]+)::[a-zA-Z1-9]+:?:?.*"))
df['theme_page2'] = df['url'].str.extract(("[a-zA-Z1-9]+::([a-zA-Z1-9]+):?:?.*"))
df.loc[df.theme_page1.isnull(),"theme_page1"] = df.loc[df.theme_page1.isnull()].url.str.extract((".*/(homes/.*)/.*"))
df.loc[df.theme_page1.isnull(),"theme_page1"] = df.loc[df.theme_page1.isnull()].url
df.theme_page1 = df.theme_page1.str.lower()
df.theme_page2 = df.theme_page2.str.lower()
df["nb_page_session"] = df.groupby(["fileid","id_client","id_session"]).id_session.transform("count")
df["duree_session"] = df.groupby(["fileid", "id_client","id_session"]).temps_passe_sur_la_page.transform("sum")
df["position_de_la_page"] = df.groupby(["fileid", "id_client","id_session"]).position_de_la_page.transform("max")


In [10]:
df

,index,fileid,id_site,id_client,id_session,position_de_la_page,niveau_2,url,temps_passe_sur_la_page,theme_page1,theme_page2,nb_sessions,duree_session,nb_page_session
40901,40901,_pages_20150601.csv,492987,6,2299141,483,Non classés,172.22.25.107/v6/edito/homes/gestion/edition.php,60,homes/gestion,NaN,483,51221,483
40902,40902,_pages_20150601.csv,492987,6,2299141,483,Non classés,172.22.25.107/v6/edito/homes/gestion/edition.php,10,homes/gestion,NaN,483,51221,483
40903,40903,_pages_20150601.csv,492987,6,2299141,483,Non classés,172.22.25.107/v6/edito/homes/gestion/edition.php,195,homes/gestion,NaN,483,51221,483
40904,40904,_pages_20150601.csv,492987,6,2299141,483,Non classés,172.22.25.107/v6/edito/homes/gestion/edition.php,27,homes/gestion,NaN,483,51221,483
40905,40905,_pages_20150601.csv,492987,6,2299141,483,Non classés,172.22.25.107/v6/edito/homes/gestion/edition.php,19,homes/gestion,NaN,483,51221,483
40906,40906,_pages_20150601.csv,492987,6,2299141,483,Non classés,172.22.25.107/v6/edito/homes/gestion/edition.php,7,homes/gestion,NaN,483,51221,483
40907,40907,_pages_20150601.csv,492987,6,2299141,483,Non classés,172.22.25.107/v6/edito/homes/gestion/edition.php,17,homes/gestion,NaN,483,51221,483
40908,40908,_pages_20150601.csv,492987,6,2299141,483,Non classés,172.22.25.107/v6/edito/homes/gestion/edition.php,8,homes/gestion,NaN,483,51221,483
40909,40909,_pages_20150601.csv,492987,6,2299141,483,Non classés,172.22.25.107/v6/edito/homes/gestion/edition.php,15,homes/gestion,NaN,483,51221,483
40910,40910,_pages_20150601.csv,492987,6,2299141,483,Non classés,172.22.25.107/v6/edito/homes/gestion/edition.php,23,homes/gestion,NaN,483,51221,483


### Tests page

In [142]:
df.niveau_2 = df.niveau_2.str.lower()
df.niveau_2 = df.niveau_2.str.normalize('NFKD')
df.niveau_2 = df.niveau_2.str.encode('ASCII', 'ignore')

In [158]:
print df.loc[(df.theme_page1.notnull())].theme_page1.value_counts(dropna=False)

general                                                                                                                                                                             6674638
General                                                                                                                                                                             4935444
Football                                                                                                                                                                            2394352
RG                                                                                                                                                                                  2377342
Tennis                                                                                                                                                                              2098531
breve                                                       

### Group by

In [ ]:
#df_sub_test['provider'] = df_sub_test['UKI'].str.extract(('.*@(.+)\..*'))
#df_compte['provider'] = df_compte['EmailAddress'].str.extract(('.*@(.+)\..*'))
df_pages_juin_p1['theme_page'] = df_pages_juin_p1['url'].str.extract(("([a-zA-Z]+::[a-zA-Z]+):?:?.*"))

In [ ]:
df_session_juin

In [ ]:
#clean datetime
#df_commande_clean['OrderDate'] = pd.to_datetime(df_commande_clean['OrderDate'])
#df_compte_clean['LastUpdated'] = pd.to_datetime(df_compte_clean['LastUpdated'])
#df_compte_clean['CreateDate'] = pd.to_datetime(df_compte_clean['CreateDate'])


df_session_juin['debut_de_session'] = pd.to_datetime(df_session_juin['debut_de_session'])
df_session_juin['fin_de_session'] = pd.to_datetime(df_session_juin['fin_de_session'])


#df_client_clean.premiere_v = pd.to_datetime(df_client_clean['premiere_v'])
#df_client_clean.derniere_v = pd.to_datetime(df_client_clean['derniere_v'])

#
df_session_juin["temps_session"] =  df_session_juin["fin_de_session"] - df_session_juin["debut_de_session"]

In [ ]:
#création de différents indicateurs
df_session_juin.loc[df_session_juin['localisation_3eme_niveau'] == 'Paris', 'Paris'] = 1
df_session_juin.loc[df_session_juin['localisation_3eme_niveau'] != 'Paris', 'Paris'] = 0

df_session_juin.loc[df_session_juin['localisation_2eme_niveau'] == 'Ile-de-France', 'RP'] = 1
df_session_juin.loc[df_session_juin['localisation_2eme_niveau'] != 'Ile-de-France', 'RP'] = 0

df_session_juin.loc[df_session_juin['localisation_1er_niveau'] == 'France', 'France'] = 1
df_session_juin.loc[df_session_juin['localisation_1er_niveau'] != 'France', 'France'] = 0


df_session_juin.loc[(df_session_juin['debut_de_session'] < date(2015, 6, 7)), 'juin_semaine_1'] = 1
#df_session_juin_clean.loc[(df_session_juin_clean['debut_de_session'] > date(2015, 6, 7)), 'juin_semaine_1'] = 0

df_session_juin.loc[(df_session_juin['debut_de_session'] < date(2015, 6, 14) ) 
                          & (df_session_juin['debut_de_session'] > date(2015, 6, 6)), 'juin_semaine_2'] = 1
#df_session_juin_clean.loc[(df_session_juin_clean['debut_de_session'] < date(2015, 6, 7) ) 
#                          & (df_session_juin_clean['debut_de_session'] > date(2015, 5, 31)), 'juin_semaine_1'] = 1

df_session_juin.loc[(df_session_juin['debut_de_session'] < date(2015, 6, 21) ) 
                          & (df_session_juin['debut_de_session'] > date(2015, 6, 13)), 'juin_semaine_3'] = 1

df_session_juin.loc[(df_session_juin['debut_de_session'] > date(2015, 6, 20) ), 'juin_semaine_4'] = 1


In [ ]:
df_session_juin

In [ ]:
#travail sur les dates des clients : 
df_client_clean["anciennete"] = (date(2015, 6, 30) - df_client_clean["premiere_v"])
df_client_clean["derniere_visite"] = (date(2015, 6, 30) - df_client_clean["derniere_v"])
df_client_clean.anciennete = pd.to_numeric(df_client_clean.anciennete)/(1000000000 * 60 * 60 * 24)
df_client_clean.derniere_visite = pd.to_numeric(df_client_clean.derniere_visite)/(1000000000 * 60 * 60 * 24)

In [ ]:
plt.figure()
#df_session_juin["debut_de_session"].groupby(df_session_juin["debut_de_session"].dt.hour).count().plot(kind="bar")
df_session_juin["debut_de_session"].groupby(df_session_juin["debut_de_session"].dt.hour).count().plot(kind="bar")
plt.savefig("debut_de_session_juin2015.png")

In [ ]:
df_commande.rename(columns = {'Somme_NetAmmount':'Somme_NetAmount'}, inplace = True)

In [ ]:
print (df_pages_juin_p1.theme_page.value_counts(dropna=False))

### Dummies

In [ ]:
#clients
#commande
#df_commande_clean_d = pd.get_dummies(df_commande_clean, columns =['theme_article'], prefix ='theme' )
#df_client_clean = pd.get_dummies(df_client_clean, columns =['id_site'], prefix ='site' )



#df_prod42_clean = pd.get_dummies(df_prod42_clean, columns =['SubscriptionStatus'], prefix ='SubStatus' )
df_prod42_clean = pd.get_dummies(df_prod42_clean, columns =['AutoRenew'], prefix ='AutoRenew' )

In [ ]:
df_session_juin

In [ ]:
print len(df_prod42.SubscriptionId.unique())
#print len( df_prod42_clean.loc[df_prod42_clean[''])

### Save

In [ ]:
df_session_juin.to_pickle(path+"\s_dataframe_sessions_juin_clean.pkl")

#si on veut sauver les dataframe en fichier excel :
#writer = pd.ExcelWriter('df_sessions_juin.xlsx', engine='xlsxwriter')
#df_session_juin.head(2).to_excel(writer, sheet_name='Sheet1')
#writer.save()

### Différence des abonnés dans comande et dans sub

In [ ]:
s=set(liste_nouveaux_sub_commande_2015)
t=set(liste_nouveaux_sub_2015)
print "taille de sub commande" , len(s)
print "taille de sub", len(t)
print "commande est il subset de sub ?", s.issubset(t)#	s <= t	test whether every element in s is in t
print "commande est il superset de sub ?", s.issuperset(t)#	s >= t	test whether every element in t is in s
print "taille de l'intersection", len(s.intersection(t))#	s & t	new set with elements common to s and t
print "elements dans commande mais pas dans sub" , len(s.difference(t))# 	s - t	new set with elements in s but not in t
print "elements dans sub mais pas dans commande" , len(t.difference(s))
#s.symmetric_difference(t)

In [ ]:
liste_nouveaux_sub_commande_2015_2 = df_commande.ClientUserId.unique()
liste_nouveaux_sub_2015_2 = df_prod42_test.ClientUserId.unique()
s=set(liste_nouveaux_sub_commande_2015_2)
t=set(liste_nouveaux_sub_2015_2)
print "taille de sub commande : " , len(s)
print "taille de sub : ", len(t)
print "commande est il subset de sub ?", s.issubset(t)#	s <= t	test whether every element in s is in t
print "commande est il superset de sub ?", s.issuperset(t)#	s >= t	test whether every element in t is in s
print "taille de l'intersection : ", len(s.intersection(t))#	s & t	new set with elements common to s and t
print "elements dans commande mais pas dans sub : " , len(s.difference(t))# 	s - t	new set with elements in s but not in t
print "elements dans sub mais pas dans commande : " , len(t.difference(s))
print "\n"

liste_nouveaux_sub_commande_juin_2015_2 = df_comm_juin.ClientUserId.unique()
liste_nouveaux_sub_juin_2015_2 = nouveaux_sub_juin.ClientUserId.unique()

s=set(liste_nouveaux_sub_commande_juin_2015_2)
t=set(liste_nouveaux_sub_juin_2015_2)
print "taille de sub commande : " , len(s)
print "taille de sub : ", len(t)
print "commande est il subset de sub ?", s.issubset(t)#	s <= t	test whether every element in s is in t
print "commande est il superset de sub ?", s.issuperset(t)#	s >= t	test whether every element in t is in s
print "taille de l'intersection : ", len(s.intersection(t))#	s & t	new set with elements common to s and t
print "elements dans commande mais pas dans sub : " , len(s.difference(t))# 	s - t	new set with elements in s but not in t
print "elements dans sub mais pas dans commande : " , len(t.difference(s))

#s.symmetric_difference(t)

### Préparation Kick-off

In [7]:
dfp = pd.read_pickle(path+"\s_dataframe_general_vel_PROD42.pkl")
dfp['SubscriptionCreated'] = pd.to_datetime(dfp['SubscriptionCreated'], format='%d/%m/%Y %H:%M:%S')
dfp['SubscriptionLastUpdated'] = pd.to_datetime(dfp['SubscriptionLastUpdated'], format='%d/%m/%Y %H:%M:%S')
dfp['ServiceExpiry'] = pd.to_datetime(dfp['ServiceExpiry'], format='%d/%m/%Y %H:%M:%S')
print 'ensemble des données prod42 :'
print "min sub creation" ,dfp.SubscriptionCreated.min()
print "max sub creation" ,dfp.SubscriptionCreated.max()
print 'min last update',dfp.SubscriptionLastUpdated.min()
print 'max last update' ,dfp.SubscriptionLastUpdated.max()
print '\n'
print "sites de l'équipe pour prod42 :"
dfp = dfp.loc[dfp['ClientUserId'].isin(liste_client)]
print "min sub creation" ,dfp.SubscriptionCreated.min()
print "max sub creation" ,dfp.SubscriptionCreated.max()
print 'min last update',dfp.SubscriptionLastUpdated.min()
print 'max last update' ,dfp.SubscriptionLastUpdated.max()

ensemble des données prod42
min sub creation 2014-04-23 15:53:59
max sub creation 2015-07-14 23:05:09
min last update 2014-07-21 00:00:19
max last update 2015-07-14 23:47:55


sites de l'équipe pour prod42
min sub creation 2014-04-28 09:52:00
max sub creation 2015-07-14 23:05:09
min last update 2014-07-21 00:00:19
max last update 2015-07-14 23:39:33


In [35]:
dfc = pd.read_pickle(r"W:\L_equipe\s_dataframe_general_xiti_clients.pkl")
dfc['date'] = dfc['fileid'].str.extract(('.+_(.+)\.csv'))

print 'ensemble des données client :'
print "min fileid" ,dfc.date.min()[:4]+"-"+dfc.date.min()[4:6]+"-"+dfc.date.min()[6:]
print "max fileid" ,dfc.date.max()[:4]+"-"+dfc.date.max()[4:6]+"-"+dfc.date.max()[6:]
print '\n'
print "sites de l'équipe pour client :"
dfc = dfc.loc[dfc['id_client'].isin(liste_client)]
print "min fileid" ,dfc.date.min()[:4]+"-"+dfc.date.min()[4:6]+"-"+dfc.date.min()[6:]
print "max fileid" ,dfc.date.max()[:4]+"-"+dfc.date.max()[4:6]+"-"+dfc.date.max()[6:]

ensemble des données client :
min fileid 2013-12-03
max fileid 2015-06-30


sites de l'équipe pour client :
min fileid 2013-12-03
max fileid 2015-06-30


Pour les sites de l'equipe <br/>
date de premiere visite min 2013-12-02 00:00:00<br/>
date de premiere visite max 2015-06-30 00:00:00

In [32]:
#dfco= pd.read_pickle(path+"\s_dataframe_general_vel_commande.pkl")
dfco['OrderDate'] = pd.to_datetime(dfco['OrderDate'], format='%d/%m/%Y %H:%M:%S')
print 'ensemble des données commande : '
print "min orderdate", dfco.OrderDate.min()
print 'max orderdate', dfco.OrderDate.max()
print '\n'
print "sites de l'équipe pour commande : "
dfco = dfco.loc[dfco['ClientUserId'].isin(liste_client)]
print "min fileid" ,dfco.OrderDate.min()
print "max fileid" ,dfco.OrderDate.max()


ensemble des données commande : 
min orderdate 2015-04-07 00:08:52
max orderdate 2015-07-05 23:56:57


sites de l'équipe pour commande : 
min fileid 2015-04-07 00:08:52
max fileid 2015-07-05 23:56:57


ensemble des données pages<br/>
date min : 2013-09-09<br/>
date max : 2015-07-08

ensembe des données sessions<br/>
date min : 2013-09-09<br/>
date max : 2015-06-30<br/>


In [45]:
dfp= pd.read_pickle(path+"\s_dataframe_general_vel_souscription.pkl")
dfp['SubscriptionCreated'] = pd.to_datetime(dfp['SubscriptionCreated'], format='%d/%m/%Y %H:%M:%S')
dfp['SubscriptionLastUpdated'] = pd.to_datetime(dfp['SubscriptionLastUpdated'], format='%d/%m/%Y %H:%M:%S')
dfp['ServiceExpiry'] = pd.to_datetime(dfp['ServiceExpiry'], format='%d/%m/%Y %H:%M:%S')
print 'ensemble des données souscription :'
print "min sub creation" ,dfp.SubscriptionCreated.min()
print "max sub creation" ,dfp.SubscriptionCreated.max()
print 'min last update',dfp.SubscriptionLastUpdated.min()
print 'max last update' ,dfp.SubscriptionLastUpdated.max()
print '\n'
print "sites de l'équipe pour souscription :"
dfp = dfp.loc[dfp['ClientUserId'].isin(liste_client)]
print "min sub creation" ,dfp.SubscriptionCreated.min()
print "max sub creation" ,dfp.SubscriptionCreated.max()
print 'min last update',dfp.SubscriptionLastUpdated.min()
print 'max last update' ,dfp.SubscriptionLastUpdated.max()

ensemble des données souscription :
min sub creation 2014-09-20 21:02:27
max sub creation 2015-07-05 21:53:22
min last update 2015-04-07 00:01:32
max last update 2015-07-05 23:56:58


sites de l'équipe pour souscription :
min sub creation 2014-09-20 21:02:27
max sub creation 2015-07-05 21:46:22
min last update 2015-04-07 00:01:32
max last update 2015-07-05 23:56:58
